In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np


from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pickle
from tensorflow import keras

# Data and Model prep

In [2]:
data_directory = "data/HMDA/"

X_test = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_Xtest.bz2')
y_test = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_ytest.bz2')
X_train = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_Xtrain.bz2')
y_train = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_ytrain.bz2')

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(650877, 35)
(650877, 1)
(278948, 35)
(278948, 1)


In [3]:
def load_model(model_type):

    if model_type == 'dt':
        model = pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_DT_scaling_20210205_014819.pkl', 'rb'))
    elif model_type == 'gbc':
        model = pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_GBC_scaling_20210205_014418.pkl', 'rb'))
    elif model_type == 'lr':
        model = pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_LR_scaling_20210205_012956.pkl', 'rb'))
    elif model_type == 'rf':
        model =  pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_RF_scaling_20210205_013239.pkl', 'rb'))
    else:
        model = keras.models.load_model(data_directory+'HMDA-MORTGAGE-APPROVAL_MLP_scaling_20210205_011811.h5')
        
    return model

# Attack

In [ ]:
from uret.utils.config import process_config_file

cf = "configs/HMDA/simanneal.yml"
num_samples = 1000
scaler = StandardScaler().fit(X_train)

def feature_extractor(x):
    if len(np.shape(x)) == 2:
        return np.array(scaler.transform(x))
    else:
        return np.array(scaler.transform([x]))

x_transformed = scaler.transform(X_test)

for mt in ['dt', 'gbc', 'lr', 'rf']:
    print("Model type:", mt)
    model = load_model(mt)
    
    model_preds = np.argmax(model.predict_proba(x_transformed),axis=1)
    
    zero_inds = np.where(model_preds == 0)[0][:num_samples]
    one_inds = np.where(model_preds == 1)[0][:num_samples]
    
    task_samples = np.concatenate((list(X_test.values[zero_inds]), list(X_test.values[one_inds])))
    
    orig_model_preds = np.argmax(model.predict_proba(feature_extractor(task_samples)),axis=1)
    
    explorer = process_config_file(cf, model, feature_extractor=feature_extractor, input_processor_list=[])
    adv_samples = explorer.explore(task_samples)  
    
    adv_model_preds = np.argmax(model.predict_proba(feature_extractor(adv_samples)),axis=1)
    
    success_rate = np.sum(adv_model_preds != orig_model_preds)/len(orig_model_preds)
    
    print("Success rate:", success_rate)
    print()
    
    pickle.dump([task_samples, adv_samples], open('data/HMDA/simanneal_adv_samples_' + mt +'.p', 'wb'))

Model type: dt


  0%|          | 0/2000 [00:00<?, ?it/s] Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00330        -13.65     0.00%     0.00%     0:00:05     0:00:37 Temperature        Energy    Accept   Improve     Elapsed   Remaining
  6%|▌         | 114/2000 [52:13<14:36:09, 27.87s/it]00:28     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining


 12%|█▏        | 235/2000 [1:49:02<12:54:34, 26.33s/it]:26     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining


 12%|█▏        | 247/2000 [1:54:19<13:09:35, 27.03s/it]:28     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.57619        -12.07    46.39%    24.17%     0:00:11     0:00:16IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 17%|█▋        | 340/2000 [2:37:43<12:47:55, 27.76s/it]:28     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.07964        -13.81     1.39%     0.83%     0:00:17     0:00:10IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg

 31%|███       | 622/2000 [4:47:01<10:54:09, 28.48s/it]:29     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.01692        -13.80     0.28%     0.28%     0:00:23     0:00:05IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 36%|███▌      | 716/2000 [5:30:12<9:42:12, 27.21s/it]0:27     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.14686        -11.52    66.11%    33.61%     0:00:09     0:00:18IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg

...

  3%|▎         | 66/2000 [32:26<15:32:14, 28.92s/it]:00:29     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00000         -6.05     0.00%     0.00%     0:00:14     0:00:15IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  7%|▋         | 149/2000 [1:12:46<15:14:25, 29.64s/it]:30     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00000         -5.82     0.00%     0.00%     0:00:29     0:00:01IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg

 20%|██        | 406/2000 [3:17:48<13:01:49, 29.43s/it]:30     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.02095         -5.10    66.67%    35.71%     0:00:06     0:00:24IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 25%|██▍       | 491/2000 [3:59:08<12:15:25, 29.24s/it]:29     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00000         -6.65     0.00%     0.00%     0:00:18     0:00:11IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg

 37%|███▋      | 741/2000 [6:00:57<10:11:43, 29.15s/it]:29     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00000         -6.46     0.00%     0.00%     0:00:28     0:00:01IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 41%|████▏     | 826/2000 [6:42:06<9:29:23, 29.10s/it]0:29     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00006         -6.49     0.95%     0.48%     0:00:11     0:00:18IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg

 54%|█████▍    | 1076/2000 [8:43:17<7:21:36, 28.68s/it]:28     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00006         -0.01    59.05%    30.95%     0:00:11     0:00:17IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 58%|█████▊    | 1161/2000 [9:24:13<6:50:00, 29.32s/it]:30     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.00000         -0.00     0.00%     0.00%     0:00:29     0:00:01IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg

 71%|███████   | 1413/2000 [11:26:48<4:42:47, 28.90s/it]29     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining


In [ ]:
cf = "configs/HMDA/simanneal_mlp.yml"
for mt in ['mlp']:
    print("Model type:", mt)
    model = load_model(mt)
    
    model_preds = np.argmax(model.predict(x_transformed),axis=1)
    
    zero_inds = np.where(model_preds == 0)[0][:num_samples]
    one_inds = np.where(model_preds == 1)[0][:num_samples]
    
    task_samples = np.concatenate((list(X_test.values[zero_inds]), list(X_test.values[one_inds])))
    
    orig_model_preds = np.argmax(model.predict(feature_extractor(task_samples)),axis=1)
    
    explorer = process_config_file(cf, model, feature_extractor=feature_extractor, input_processor_list=[])
    adv_samples = explorer.explore(task_samples)  
    
    adv_model_preds = np.argmax(model.predict(feature_extractor(adv_samples)),axis=1)
    
    success_rate = np.sum(adv_model_preds != orig_model_preds)/len(orig_model_preds)
    
    print("Success rate:", success_rate)
    print()
    
    pickle.dump([task_samples, adv_samples], open('data/HMDA/simanneal_adv_samples_' + mt +'.p', 'wb'))